In [1]:
from math import log
from collections import Counter,defaultdict
from functools import partial

In [2]:
def entropy(class_probabilities):
    """ given a list of class proabilities, compute entropy """
    return sum(-p * log(p,2)
              for p in class_probabilities
              if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count/total_count
           for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _,label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

# entropy([0.999,0.001]) ==> 0.011407757737461138

# class_probabilities([0,0,1]) ==>
#     [0.6666666666666666, 0.3333333333333333]

In [3]:
inputs = [
    ({'level':'Senior','lang':'Python'},True),
    ({'level':'Mid','lang':'Java'},True),
    ({'level':'Junior','lang':'Python'},False),
    ({'level':'Mid','lang':'Python'},True)
         ]

In [4]:
# get the entropy of a given data partition
def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

def partition_by(inputs, attribute):
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups

def partition_entropy_by(inputs,attributes):
    partitions = partition_by(inputs, attributes)
    return partition_entropy(partitions.values())

# for key in ['level','lang']:
#     print(partition_entropy_by(inputs,key))
    
# 0.0
# 0.6887218755408672

In [14]:
def classify(tree,inp):
    if tree in [True,False]:
        return tree
    attribute,subtree_dict = tree
    
    subtree_key = inp.get(attribute)
    
    if subtree_key not in subtree_dict:
        subtree_key = None
    subtree = subtree_dict[subtree_key]
    
    return classify(subtree,inp)

def build_tree_id3(inputs, split_candidates=None):
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
        
    num_inputs = len(inputs)
    num_trues = len([label for item,label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0: return False
    if num_falses == 0: return True
    
    if not split_candidates:
        return num_trues >= num_falses
    
    best_attribute = min(split_candidates,
                        key=partial(partition_entropy_by,inputs))
    
    partitions = partition_by(inputs,best_attribute)
    new_candidates = [a for a in split_candidates
                     if a != best_attribute]
    
    subtrees = {attribute_value : build_tree_id3(subset,new_candidates)
               for attribute_value, subset in partitions.items()}
    
    subtrees[None] = num_trues > num_falses
    
    return (best_attribute,subtrees)

In [15]:
tree = build_tree_id3(inputs)

In [16]:
tree

('level', {None: True, 'Senior': True, 'Junior': False, 'Mid': True})

In [19]:
classify(tree,{'lang':'Python'})

True

In [ ]:
def forest_classify(trees,input):
    vote = [classify(tree,input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[1][0]